In [5]:
%pwd

'/Users/mukulagarwal/Desktop/Projects/datascienceproject/research'

In [6]:
%cd ..

/Users/mukulagarwal/Desktop/Projects/datascienceproject


/Users/mukulagarwal/Desktop/Projects/datascienceproject/prjt3/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
%pwd

'/Users/mukulagarwal/Desktop/Projects/datascienceproject'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
import os
from src.datascience.utils.common import read_yaml, create_directories 
from src.datascience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
class Configuration_Manager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        create_directories([self.config.data_ingestion.root_dir])
        config = self.config.data_ingestion
        
        data_ingestion = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion       

In [19]:
import urllib.request as request
from src.datascience import logger
import os
import zipfile

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info \n{headers}")
            
        else:
            logger.info(f"{self.config.local_data_file} file already exists")  
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = Configuration_Manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-28 01:23:22,867: INFO: common: yaml file at config/config.yaml loaded successfully]
[2024-10-28 01:23:22,868: INFO: common: yaml file at params.yaml loaded successfully]
[2024-10-28 01:23:22,869: INFO: common: yaml file at schema.yaml loaded successfully]
[2024-10-28 01:23:22,869: INFO: common: Directory created at artifacts]
[2024-10-28 01:23:22,870: INFO: common: Directory created at artifacts/data_ingestion]
[2024-10-28 01:23:23,635: INFO: 393514883: artifacts/data_ingestion/data.zip downloaded with following info 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2253:3D1C44:5BC63:72893:671E9A32
Accept-Ranges: bytes
Date: Sun, 2